In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch

from torch import nn
torch.cuda.empty_cache()

import sys
sys.path.append('../')
from model_runner import ModelRunner
from image_helper import ImageHelper
from SRCNN_different_specs import SRCNN
from SRCNN import SRCNN
from custom_image_dataset import CustomImageDataset

sys.path.append('../DBPN/')
from DBPN_copied import DBPN


/home/pedrovelozo/TCC-Pedro-Velozo-ENG1132/results/../model_runner.py:307: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if model_name is "Bicubic":


In [2]:
DATASET_ROOT="../datasets/"
DATASET_NAME = "set14"
# DATASET_NAME = "Set5"
FULL_DATASET_PATH = f"{DATASET_ROOT}{DATASET_NAME}"
model_runner = ModelRunner()
image_helper = ImageHelper()

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load models
srcnn = SRCNN(f2=5).to(device)
dbpn = more_epochs_less_data_model = DBPN(num_channels=3, base_filter=64,  feat = 256, num_stages=7, scale_factor=4).to(device)

try:
    srcnn.load_state_dict(torch.load('./srcnn/trained_models/model_f2_5.pth'))
    dbpn.load_state_dict(torch.load('./dbpn/trained_models/model-05-08-epoch=0-299.pth'))

except:
    srcnn.load_state_dict(torch.load('./srcnn/trained_models/model_f2_5.pth', map_location=torch.device('cpu')))
    dbpn.load_state_dict(torch.load('./dbpn/trained_models/model-05-08-epoch=0-299.pth', map_location=torch.device('cpu')))


In [8]:
model_runner.compare_models([
                              {"name": "SRCNN", "model": srcnn, "scale": 3},
                              {"name": "DBPN", "model": dbpn,
                               "input_transform":lambda input: transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Resize(
                                    (input.size[1] // 4, input.size[0] // 4), interpolation=Image.BICUBIC),
                                ])(input),
                                "should_only_downsampple": True, "scale":4, "unsqueeze": True },
                              ], FULL_DATASET_PATH, scale=4)

0it [00:00, ?it/s]/home/pedrovelozo/TCC-Pedro-Velozo-ENG1132/env/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/pedrovelozo/TCC-Pedro-Velozo-ENG1132/env/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `P

,PSNR,SSIM
SRCNN,24.815783,0.726430
DBPN,23.480452,0.660932
Bicubic,22.933111,0.659410


In [ ]:
print(656)
print(656 // 4)
print((656 // 4)*4)

print()

print(529)
print(529 // 4)
print((529 // 4)*4)


In [ ]:
a = torch.randn([3, 128, 192])
dbpn(a.unsqueeze(0).to(device)).shape

In [ ]:
b = torch.randn([1, 3, 128, 192])
dbpn(b.to(device)).shape

In [ ]:
IMAGE_EXAMPLE = "../datasets/set14/ppt3.png"

image_helper.show_tensor_as_images_side_by_side(
    [
        {
            "label": "Low Res",
            "tensor": image_helper.downsample_image_as_tensor(
                IMAGE_EXAMPLE, 4, interpolation=Image.BICUBIC,
            ),
        },
        {
            "label": "DBPN",
            "tensor": image_helper.apply_model_to_image(
                model=dbpn,
                image=IMAGE_EXAMPLE,
                downsample_factor=4,
                should_upsample=False,
                unsqueeze=True
            )
        },
    ],
)

In [ ]:
image = Image.open('../datasets/set14/ppt3.png')

a = image_helper.downsample_image_as_tensor(image, 4, interpolation=Image.BICUBIC, unsqueeze=True)
print(image.size)
print(a.shape)
b = dbpn(a.to(device))
print(b.shape)

# a = image_helper.crop_image_to_nearest_even_dimensions_and_transform_to_tensor(image)

# print(image.size)
# print(a.shape)
# image_center_crop = transforms.CenterCrop((image.size[1], image.size[0]-1))(image)
# image_tensor = transforms.ToTensor()(image)

# print(image_center_crop.size)
# image_center_crop
# print(image_tensor.shape)

# a = dbpn(image_tensor.unsqueeze(0).to(device))


In [ ]:
dbpn(image)

In [ ]:
592 // 4 * 4